In [1]:
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)

In [ ]:
import os
os.chdir('simulation/Sparse_Effects_Setting')
os.getcwd()

# Data Simulation

In [3]:
## strong confoundedness: large B and gamma
## small true effect: small tau

k = 500
s = 3
np.random.seed(123)
B = np.random.uniform(low=-2, high=2, size=(k,s))
# B  = np.zeros(shape = (k,s))
# B = np.array([2, 0.5, -0.4, 0.2]).reshape(k, s)
sigma2_tstar = 100

gamma = np.array([100]*s).reshape(s,1)
tau = np.random.uniform(low=-0.1, high=0.1, size=(k,1))
sigma2_y = 10

# set some tau to large values #
nontrivial_effect_index = np.random.randint(low = 0, high = k, size = int(k*0.1))
nontrivial_effect_index = np.unique(nontrivial_effect_index)
nontrivial_effect = np.random.uniform(low=-2, high=2, size=(nontrivial_effect_index.shape[0],1))
tau[nontrivial_effect_index] = nontrivial_effect

In [4]:
n = 50000
u = np.random.multivariate_normal(mean = np.full(s, 0), cov=np.identity(s), size=n)
tr_star = np.dot(u, np.transpose(B)) + \
          np.random.multivariate_normal(mean = np.full(k, 0), cov=sigma2_tstar*np.identity(k), size = n)
tr = np.where(tr_star > 0, 1, 0)
y = np.dot(tr, tau) + np.dot(u, gamma) + np.random.normal(loc=0, scale=np.sqrt(sigma2_y), size=n).reshape(n,1)
# y = np.dot(tr_star, tau) + np.dot(u, gamma) + np.random.normal(loc=0, scale=np.sqrt(sigma2_y), size=n)

In [5]:
## the first eigenvalue of BB is large, thus var(u|t) is smaller given sigma2_star=100
BB = np.dot(B, np.transpose(B))
BB_w, BB_v = np.linalg.eig(BB) ## eigenvalues, eigevector
BB_w[0:10] ## the first eigenvalue, 654.66

array([754.59642144+0.j, 694.21036223+0.j, 606.69565787+0.j,
        -0.        +0.j,  -0.        -0.j,   0.        +0.j,
         0.        +0.j,   0.        -0.j,  -0.        +0.j,
        -0.        -0.j])

In [6]:
## some theoretical values #
coef_mu_u_tstar = np.dot(np.transpose(B), np.linalg.inv(np.dot(B, np.transpose(B)) + sigma2_tstar*np.identity(k)))
cov_u_tstar = np.identity(s) - np.dot(coef_mu_u_tstar, B)
print('Cov(u|t*):')
print(cov_u_tstar)

Cov(u|t*):
[[ 0.12583743  0.01137063 -0.00124313]
 [ 0.01137063  0.13200804 -0.00338941]
 [-0.00124313 -0.00338941  0.12658369]]


In [7]:
var_y_t = np.dot(np.dot(np.transpose(gamma), cov_u_tstar), gamma) + sigma2_y
print('Approx. Var(y|t) [Var(y|t*)]: ' + str(var_y_t[0,0]))
print('confounding fraction in residual of Y: ' + str(1 - sigma2_y/var_y_t[0,0]))

Approx. Var(y|t) [Var(y|t*)]: 3989.0533932258177
confounding fraction in residual of Y: 0.9974931395962305


In [8]:
# True Treatment Effect
tau.reshape(k,)

array([-0.0079136 , -0.08092806,  0.08204695,  0.03360369,  0.00199419,
       -0.01374685, -0.0897069 , -0.05714116,  0.07121584,  0.02444088,
       -0.06122498, -0.06215936, -0.0299749 ,  0.02305931,  0.00848305,
        0.00540698,  0.07378616,  0.07225029,  0.0690397 ,  0.04049034,
       -0.02150996, -0.04613115,  0.09113853, -0.09094301,  0.02367661,
        0.06922482, -0.04561742,  0.09574921,  0.07718479, -0.06013438,
       -0.04593814,  0.07946058, -0.07814964, -0.06230457,  0.22737075,
       -0.07059713,  0.01450903,  0.01510904,  0.01927088, -0.04985972,
        0.06421617,  0.09408872, -0.0444228 ,  0.07309866, -0.04138959,
       -0.03021458,  0.05708869, -1.72425897, -0.08763849, -0.02267348,
       -0.07406336, -0.08408945,  0.0177456 , -0.06320505, -1.33897426,
        0.04581771, -0.08140086,  0.09879365, -0.03824867,  0.00912585,
        0.0583699 ,  0.92088713, -0.03176725, -0.05874625, -0.02428434,
        0.04698733,  0.04288538, -0.06359616, -0.78220437,  0.03

In [9]:
# Approx. Obs. Effect #
effect_bias = np.dot(np.dot(np.transpose(gamma), coef_mu_u_tstar), \
                     np.identity(k)).reshape(k,)
print('effect bias range:')
print(effect_bias.min())
print(effect_bias.max())
effect_obs = tau.reshape(k,) + effect_bias
print('obs effect range:')
print(effect_obs.min())
print(effect_obs.max())

effect bias range:
-0.7511657117030033
0.7201117743876483
obs effect range:
-2.1567691647786456
1.860814044224537


In [74]:
print('Effect Bias for Nontrivial Ones:     ' + str(effect_bias[nontrivial_effect_index]))
print('Observed Effect for Nontrivial Ones: ' + str(effect_obs[nontrivial_effect_index]))
print('True Effect for Nontrivial Ones:     ' + str(tau[nontrivial_effect_index].reshape(50,)))

Effect Bias for Nontrivial Ones:     [-0.19156168 -0.00601164  0.50798291 -0.16353192 -0.13218608  0.07048387
 -0.25363315 -0.17817511  0.03897286 -0.44606751 -0.08022859  0.1626099
 -0.21195302  0.31633593 -0.17574154 -0.387714   -0.43710003 -0.22362933
  0.02218435 -0.43710003  0.02260092  0.0250833  -0.20360345  0.29173772
 -0.35123563 -0.29887068  0.17314676  0.29568091 -0.11594583 -0.13336539
  0.01345059  0.29568091 -0.59743655 -0.41687289 -0.30065269 -0.09341714
 -0.09134576 -0.10699182 -0.32643323 -0.50785215  0.37873456 -0.02648453
 -0.3792888   0.30177889  0.16160004 -0.16403011 -0.23671804  0.1332102
 -0.15640847  0.38417182]
Observed Effect for Nontrivial Ones: [ 0.03580907 -1.73027062 -0.83099135  0.75735521 -0.91439045 -1.19905462
 -2.05937236 -1.89784424 -0.55249467 -2.39413616  0.69515799  2.11684109
 -1.86499218  1.02549347 -0.95280136 -0.38364411 -2.20705234  1.4174158
  0.16782274 -2.20705234 -1.44307753 -1.0034953  -1.19302549 -0.93953058
  0.48178681  0.37904891  0

In [3]:
# pd.DataFrame(tau).to_csv('tau.csv', index = False)
# pd.DataFrame(u).to_csv('u.csv', index = False)
# pd.DataFrame(tr).to_csv('tr.csv', index = False)
# pd.DataFrame(y).to_csv('y.csv', index = False)
# pd.DataFrame(nontrivial_effect_index).to_csv('nontrivial_effect_index.csv', index = False)
n = 50000
u = pd.read_csv('u.csv').to_numpy()
tr = pd.read_csv('tr.csv').to_numpy()
y = pd.read_csv('y.csv').to_numpy()
tau = pd.read_csv('tau.csv').to_numpy()
nontrivial_effect_index = pd.read_csv('nontrivial_effect_index.csv').to_numpy().reshape(50,)

# VAE

In [ ]:
import os
os.chdir('simulation/Sparse_Effects_Setting/LatentDim3')
os.getcwd()

In [14]:
from keras.layers import Input, Dense, Lambda, LeakyReLU
from keras.models import Model
from keras import backend as K
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.optimizers import RMSprop
from keras import initializers
from keras import regularizers
from keras import optimizers
import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


In [15]:
batch_size = 1000
original_dim = k
latent_dim = s
intermediate_dim = 10
epochs = 300
epsilon_std = 1
z_log_sigma_prior = np.log(0.5)

In [16]:
# encoder #
x = Input(batch_shape=(batch_size, original_dim))
h = Dense(intermediate_dim)(x)
h = LeakyReLU(alpha = 0.1)(h)
z_mean = Dense(latent_dim)(h)
# z_log_sigma = Dense(latent_dim)(h)
## log_z_scale


z_log_sigma_input = Input(batch_shape = (batch_size, 1))
z_log_sigma = Dense(units = 1,  activation = "linear",
                    kernel_initializer=initializers.Ones(),
                    use_bias = False)(z_log_sigma_input)

In [17]:
# sampling from latent space #
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim))
    return z_mean + K.exp(z_log_sigma) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
# so you could write `Lambda(sampling)([z_mean, z_log_sigma])`
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])
# z = Lambda(sampling)([z_mean, z_log_sigma])

In [18]:
# decoder #
decoder_h1 = Dense(intermediate_dim)
decoder_h2 = LeakyReLU(alpha = 0.1)
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h1(z)
h_decoded = decoder_h2(h_decoded)
x_decoded_mean = decoder_mean(h_decoded)

In [19]:
# end-to-end autoencoder
vae = Model([x, z_log_sigma_input], x_decoded_mean)

# encoder, from inputs to latent space
encoder_z_mean = Model([x, z_log_sigma_input], z_mean)
encoder_z_log_sigma = Model([x, z_log_sigma_input], z_log_sigma)

# generator, from latent space to reconstructed inputs
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h1(decoder_input)
_h_decoded = decoder_h2(_h_decoded)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = Model(decoder_input, _x_decoded_mean)

In [20]:
def vae_loss(x, x_decoded_mean):
    xent_loss = original_dim * binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    return xent_loss + kl_loss

In [21]:
def recon_metric(x, x_decoded_mean):
    xent_loss = original_dim * binary_crossentropy(x, x_decoded_mean)
    return xent_loss

In [22]:
opt = optimizers.RMSprop(learning_rate=0.0005)
# opt = optimizers.Adam(learning_rate=0.001)
vae.compile(optimizer=opt, loss=vae_loss, metrics = [recon_metric])

In [23]:
from sklearn.model_selection import train_test_split
tr_train, tr_test = train_test_split(tr,train_size=0.8)
log_sigma_input_train = np.full((tr_train.shape[0], 1), z_log_sigma_prior)
log_sigma_input_test = np.full((tr_test.shape[0], 1), z_log_sigma_prior)

In [24]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
from keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor='val_loss', patience=3)

In [25]:
# fitting VAE #
vae.fit([tr_train,log_sigma_input_train], tr_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=([tr_test, log_sigma_input_test], tr_test),
        callbacks = [callback])

Train on 40000 samples, validate on 10000 samples
Epoch 1/300
40000/40000 [==============================] - 2s 49us/step - loss: 346.8502 - recon_metric: 346.6636 - val_loss: 346.7291 - val_recon_metric: 346.6055
Epoch 2/300
40000/40000 [==============================] - 1s 29us/step - loss: 346.6751 - recon_metric: 346.5442 - val_loss: 346.5910 - val_recon_metric: 346.4373
Epoch 3/300
40000/40000 [==============================] - 1s 30us/step - loss: 346.4131 - recon_metric: 346.1924 - val_loss: 346.1993 - val_recon_metric: 345.9050
Epoch 4/300
40000/40000 [==============================] - 1s 30us/step - loss: 345.8714 - recon_metric: 345.4674 - val_loss: 345.5515 - val_recon_metric: 345.0258
Epoch 5/300
40000/40000 [==============================] - 1s 29us/step - loss: 345.1980 - recon_metric: 344.6090 - val_loss: 344.8254 - val_recon_metric: 344.1248
Epoch 6/300
40000/40000 [==============================] - 1s 31us/step - loss: 344.4634 - recon_metric: 343.7009 - val_loss: 344.

In [27]:
log_sigma_input = np.full((tr.shape[0], 1), z_log_sigma_prior)
u_t_mean = encoder_z_mean.predict([tr, log_sigma_input], batch_size = batch_size)
u_t_mean

array([[-0.65206206, -0.6052956 , -0.8986118 ],
       [-0.65482724,  0.9320263 ,  1.2972934 ],
       [-0.87104845, -2.4526317 ,  0.8720732 ],
       ...,
       [-3.4867055 ,  1.6833498 , -0.5314662 ],
       [ 0.674997  , -1.3439927 ,  1.0985439 ],
       [-0.06504131, -0.5549359 ,  2.3187962 ]], dtype=float32)

In [28]:
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
u_pcs = PCA(n_components = latent_dim).fit_transform(u)
uhat_pcs = PCA(n_components = latent_dim).fit_transform(u_t_mean)
for i in range(latent_dim):
    print(i)
    print(pearsonr(u_pcs[:,i], uhat_pcs[:,i]))

0
(-0.16571916547719565, 9.999214617874603e-305)
1
(-0.5256800884366084, 0.0)
2
(-0.2116353047190557, 0.0)


In [29]:
## the coef of imput_log_sigma #
encoder_z_log_sigma.layers[1].get_weights()[0]

array([[2.1277642]], dtype=float32)

In [30]:
# the output sigma_u_t #
u_log_sigma = encoder_z_log_sigma.predict([tr_train, log_sigma_input_train], batch_size = batch_size)
# print(u_log_sigma)
u_t_sigma = np.exp(u_log_sigma)
u_t_sigma[0,0]

0.22881219

In [31]:
# the output sigma_u_t #
np.exp(z_log_sigma_prior*encoder_z_log_sigma.layers[1].get_weights()[0][0][0])

0.22881218301308015

# Importance Sampling Estimates (ISE)

In [32]:
from scipy.stats import norm
from scipy.stats import multivariate_normal

In [33]:
# vae ouput sigma_u_t #
u_t_sigma = np.exp(z_log_sigma_prior*encoder_z_log_sigma.layers[1].get_weights()[0][0][0])

In [34]:
def f_t_u_ise(i):
    if (i % 1000 == 0): 
        print(i)
    nsim = 100
    t = tr[i]
    u_t_mean = encoder_z_mean.predict([t.reshape(1,k), np.array([[z_log_sigma_prior]])])
    u_samples = np.random.multivariate_normal(mean = u_t_mean[0], \
                                cov = u_t_sigma*np.identity(latent_dim), size = nsim)
    # f(t|u) #
    p_t_u = generator.predict(u_samples) ## sim in rach row
    f_t_u = pd.DataFrame(p_t_u).apply(lambda x: np.prod(x**t * (1-x)**(1-t)), axis=1) ## sim in rach row
    # f(u) #
    f_u = multivariate_normal(mean=[0]*latent_dim, cov=np.identity(latent_dim)).pdf(u_samples)
    # q(u|t) #
    q_u_t = multivariate_normal(mean = u_t_mean[0], cov = u_t_sigma*np.identity(latent_dim)).pdf(u_samples)
    # w = f(t|u)f(u)/q(u|t) #
    nsamples = nsim
    w = 10**(np.log10(f_t_u)+np.log10(f_u)-np.log10(q_u_t)).to_numpy().reshape(nsamples,1)
    weight = 10**(np.log10(w) - np.log10(w.mean()))
    mu_u_t_ise = (u_samples*weight).mean(axis=0)
    mu_uu_t_ise = (np.apply_along_axis(lambda x: np.outer(x,x),1,u_samples)*weight.reshape(nsamples,1,1)).mean(axis=0)
    cov_u_t_ise = mu_uu_t_ise - np.outer(mu_u_t_ise, mu_u_t_ise)
    #     mu_u2_t_ise = (u_samples**2 * weight).mean(axis=0)
    #     var_u_t_ise = mu_u2_t_ise - mu_u_t_ise**2
    result = np.array([u_t_mean[0], mu_u_t_ise, cov_u_t_ise, weight.reshape(len(w))], dtype=object)
    return result

In [35]:
ise_results = [f_t_u_ise(i) for i in range(tr.shape[0])]
# mu_u_t (VAE output); mu_u_t_ise, var_u_t_ise

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000


In [36]:
mu_u_t_output = np.empty((0, latent_dim))
mu_u_t_ise = np.empty((0, latent_dim))
cov_u_t_ise = np.empty((0, latent_dim, latent_dim))
for i in range(len(ise_results)):
    mu_u_t_output = np.append(mu_u_t_output, ise_results[i][0].reshape(1,latent_dim), axis = 0)
    mu_u_t_ise = np.append(mu_u_t_ise, ise_results[i][1].reshape(1,latent_dim), axis = 0)
    cov_u_t_ise = np.append(cov_u_t_ise, ise_results[i][2].reshape(1,latent_dim,latent_dim), axis = 0)
weight_mat = [ise_results[i][3] for i in range(len(ise_results))]

In [37]:
# output E(u|t) #
mu_u_t_output.mean(axis=0)

array([-0.26066986,  0.06580309,  0.26675362])

In [38]:
# ISE E(u|t) #
mu_u_t_ise.mean(axis=0)

array([-0.10913082,  0.03865301,  0.12847931])

In [39]:
# output Var(u|t) #
print(u_t_sigma**2)

0.05235501509521128


In [40]:
# ISE Cov(u|t) #
cov_u_t_ise_ave = cov_u_t_ise.mean(axis=0)
cov_u_t_ise_ave

array([[ 0.17545018,  0.0081635 , -0.00346365],
       [ 0.0081635 ,  0.1743882 , -0.00179012],
       [-0.00346365, -0.00179012,  0.1867698 ]])

In [41]:
pd.DataFrame(mu_u_t_ise).to_csv('mu_u_t_ise.csv', index = False)
pd.DataFrame(cov_u_t_ise_ave).to_csv('cov_u_t_ise.csv', index = False)
# mu_u_t_ise = pd.read_csv('mu_u_t_ise.csv').to_numpy()
# cov_u_t_ise = pd.read_csv('cov_u_t_ise.csv').to_numpy()

In [42]:
cov_u = cov_u_t_ise.mean(axis=0) + np.cov(np.transpose(mu_u_t_ise))
cov_u

array([[ 0.95062671, -0.02078696,  0.01938876],
       [-0.02078696,  0.94954662,  0.00707655],
       [ 0.01938876,  0.00707655,  0.99638468]])

In [43]:
print('ISE Cov(U|t):')
print(cov_u_t_ise_ave)
# Cov(u) = E(Cov(u|t)) + Cov(E(u|t))
cov_u = cov_u_t_ise.mean(axis=0) + np.cov(np.transpose(mu_u_t_ise))
print('Cov(U):')
print(cov_u)
print('var(u|t)/var(u) for each dimension of U')
print(np.diag(cov_u_t_ise_ave)/np.diag(cov_u))

ISE Cov(U|t):
[[ 0.17545018  0.0081635  -0.00346365]
 [ 0.0081635   0.1743882  -0.00179012]
 [-0.00346365 -0.00179012  0.1867698 ]]
Cov(U):
[[ 0.95062671 -0.02078696  0.01938876]
 [-0.02078696  0.94954662  0.00707655]
 [ 0.01938876  0.00707655  0.99638468]]
var(u|t)/var(u) for each dimension of U
[0.18456264 0.18365417 0.18744749]


# -----------------------------------------------------------------------------------------

# Fitting LM with Y ~ T

In [44]:
from sklearn.linear_model import LinearRegression
lmfit_y_t = LinearRegression().fit(tr, y)

In [45]:
# coefficient： tau_naive #
tau_naive = lmfit_y_t.coef_.reshape(k,1)  # lmfit_y_t.intercept_

In [46]:
tau_naive[nontrivial_effect_index].reshape(50,)

array([ -3.85284773,  -0.91089822,  12.09315041,  -3.25899139,
        -5.4797407 ,   0.04407081,  -7.36560428,  -6.34271326,
         1.02171412, -13.13578741,  -0.81916312,   6.95351223,
        -6.64761181,   9.55408621,  -5.28890334,  -8.35829301,
       -11.70187037,  -4.04250111,  -0.03879425, -11.70187037,
        -1.34684483,  -0.07948535,  -4.37678675,   5.5480885 ,
        -7.23402651,  -6.62003685,   4.65643001,  10.25216207,
        -4.34226198,  -3.64125522,   0.11976007,  10.25216207,
       -13.14796222,  -9.29432922,  -6.37545183,  -2.92180524,
         1.2008204 ,  -3.36721287,  -8.44000453, -13.45396435,
        10.40681793,  -1.7186441 ,  -7.6641055 ,   8.42386768,
         5.18251881,  -3.52986793,  -6.43146727,   1.00636515,
        -3.6737733 ,   8.76517495])

In [47]:
effect_obs[nontrivial_effect_index]

array([ 0.03580907, -1.73027062, -0.83099135,  0.75735521, -0.91439045,
       -1.19905462, -2.05937236, -1.89784424, -0.55249467, -2.39413616,
        0.69515799,  2.11684109, -1.86499218,  1.02549347, -0.95280136,
       -0.38364411, -2.20705234,  1.4174158 ,  0.16782274, -2.20705234,
       -1.44307753, -1.0034953 , -1.19302549, -0.93953058,  0.48178681,
        0.37904891,  0.62561858,  2.20776609, -1.10521291, -1.35561128,
       -0.21709315,  2.20776609,  0.8889774 ,  0.09308352,  0.48711883,
       -1.5722218 ,  1.79951197, -1.65310123, -1.68726251, -1.6717612 ,
        1.81350111, -1.07284413,  1.3298569 ,  1.68364053,  1.03405283,
        0.94847316, -0.23508562, -1.28451368, -0.07296996,  1.30025698])

In [48]:
y_hat = lmfit_y_t.predict(tr)
var_y_t = sum((y - y_hat)**2)/(y.shape[0]-1-tr.shape[1])

In [49]:
var_y_t

array([5888.45788407])

## Compute $E(U|Ti=1) - E(U|Ti=0)$ by raw VAE estimates

In [71]:
import copy
def cal_u_t_diff_org(i):
    print(i)
    t1 = copy.deepcopy(tr)
    t1[:,i] = 1
    u_t1 = encoder_z_mean.predict([t1, log_sigma_input], batch_size = batch_size)
    t0 = copy.deepcopy(tr)
    t0[:,i] = 0
    u_t0 = encoder_z_mean.predict([t0, log_sigma_input], batch_size = batch_size)
    return (u_t1 - u_t0).mean(axis = 0)

In [72]:
u_t_diff_org_all = [cal_u_t_diff_org(i) for i in range(tr.shape[1])]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [73]:
pd.DataFrame(u_t_diff_org_all).to_csv('u_t_diff_org_all.csv', index = False)

## Compute $E(U|Ti=1) - E(U|Ti=0)$ by ISE estimates

In [ ]:
def cal_mu_u_t_ise(t):
    nsim = 100
    u_t_mean = encoder_z_mean.predict([t.reshape(1,k), np.array([[z_log_sigma_prior]])])
    u_samples = np.random.normal(loc = u_t_mean[0,0], scale = u_t_sigma, size = nsim)
    # f(t|u) #
    p_t_u = generator.predict(u_samples.reshape(nsim,1)) ## sim in rach row
    f_t_u = pd.DataFrame(p_t_u).apply(lambda x: np.prod(x**t * (1-x)**(1-t)), axis=1) ## sim in rach row
    # f(u) #
    f_u = norm(loc=0, scale=1).pdf(u_samples)
    # q(u|t) #
    q_u_t = norm(loc=u_t_mean[0,0], scale=u_t_sigma).pdf(u_samples)
    # w = f(t|u)f(u)/q(u|t) #
    w = f_t_u*f_u/q_u_t
    mu_u_t_ise = (u_samples*w).mean() / w.mean()
    return mu_u_t_ise ## averaged (single value)

In [ ]:
import copy
def cal_u_t_diff(i):
    print(i)
    t1 = copy.deepcopy(tr)
    t1[:,i] = 1
    u_t1 = np.apply_along_axis(cal_mu_u_t_ise, 1, t1)
    t0 = copy.deepcopy(tr)
    t0[:,i] = 0
    u_t0 = np.apply_along_axis(cal_mu_u_t_ise, 1, t0)
    return (u_t1 - u_t0).mean(axis = 0)

In [ ]:
# u_t_diff_ise_all = [cal_u_t_diff(i) for i in range(tr.shape[1])]

In [ ]:
u_t_diff_ise_t1 = cal_u_t_diff(0) 
u_t_diff_ise_t2 = cal_u_t_diff(1) 
u_t_diff_ise_t3 = cal_u_t_diff(2) 
u_t_diff_ise_t4 = cal_u_t_diff(3) 

In [ ]:
print(pd.DataFrame(u_t_diff_org_all)[:4])
print(pd.DataFrame([u_t_diff_ise_t1, u_t_diff_ise_t2, \
                    u_t_diff_ise_t3, u_t_diff_ise_t4]))

In [ ]:
# pd.DataFrame(u_t_diff_all_ise).to_csv('u_t_diff_all_ise.csv', index = False)

In [ ]:
# pd.DataFrame([u_t_diff_ise_t1, u_t_diff_ise_t2, \
#               u_t_diff_ise_t3, u_t_diff_ise_t4]).to_csv('u_t_diff_ise_1234.csv', index = False)